### Author: Juan Nembaopit

## Objective

Tujuan utama pengembangan sistem ini adalah membuat asisten pajak digital (Chatbot) yang mampu menjawab pertanyaan kompleks seputar perpajakan Indonesia secara akurat menggunakan RAG dengan GPT-4 dan memastikan jawaban terstruktur tanpa referensi eksternal. Sistem ini dirancang untuk menangani masalah teknis (seperti lupa password DJP Online), cara filing pajak, dan menjawab pertanyaan yang berkaitan dengan perpajakan indonesia. Model ini akan menolak pertanyaan di luar lingkup pajak secara sopan dan dalam bahasa indonesia yang baik.

In [1]:
!pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 19.0 MB/s eta 0:00:00


In [2]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.1/434.1 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.5 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.52
    Uninstalling langchain-core-0.3.52:
      Successfully uninstalled langchain-core-0.3.52
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.23
    Uninstalling langchain-0.3.23:
      Successfully uninstalled langchain-0.3.23


In [3]:
!pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.8 MB/s eta 0:00:00


In [4]:
!pip install langchain_mongodb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 4.7 MB/s eta 0:00:00


# Proses Pengerjaan

In [32]:
from pymongo import MongoClient
from langchain_mongodb import MongoDBAtlasVectorSearch
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
import os
import re

In [33]:
# Load Environment Variables
load_dotenv()
MONGODB_URI = os.getenv("MONGODB_URI")
OPENAI_KEY = os.getenv("OPENAI_API_KEY")

In [34]:
# Initialize Embeddings
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    openai_api_key=OPENAI_KEY,
    dimensions=1536
)

In [35]:
# MongoDB Connection
client = MongoClient(MONGODB_URI)
collection = client['Astrax_db']['Astrax']

In [36]:
# Vector Store Configuration
vector_store = MongoDBAtlasVectorSearch(
    collection=collection,
    embedding=embeddings,
    index_name='vector_index',
    text_key="text"
)

In [37]:
# Optimized Professional Prompt Template
PROFESSIONAL_PROMPT = PromptTemplate(
    input_variables=["context", "question"],
    template="""
    Anda adalah Asisten Pajak Profesional Direktorat Jenderal Pajak Indonesia.
    Tugas utama:
    1. Jawab pertanyaan pajak berdasarkan FAQ resmi DJP
    2. Berikan panduan teknis pelaporan pajak dan masalah akun DJP Online
    3. Jelaskan konsep perpajakan dengan bahasa sederhana
    4. Bantu masalah teknis terkait layanan digital DJP

    Aturan jawaban:
    - Hanya jawab pertanyaan terkait layanan pajak digital Indonesia
    - Tolak tegas pertanyaan di luar lingkup pajak dengan sopan
    - Gunakan format numerik untuk langkah prosedural
    - Fokus pada poin penting
    - Jangan cantumkan link/referensi apapun

    Konteks resmi:
    {context}

    Pertanyaan: {question}

    Jawaban profesional:
    """
)

In [38]:
# Model Configuration dengan GPT-4
llm = ChatOpenAI(
    model_name="gpt-4",
    openai_api_key=OPENAI_KEY,
    temperature=0,
    max_tokens=800
)

In [39]:
# Optimized Retrieval Chain
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_store.as_retriever(
        search_type="similarity",
        search_kwargs={
            "k": 3,
            "score_threshold": 0.78
        }
    ),
    chain_type_kwargs={"prompt": PROFESSIONAL_PROMPT},
    return_source_documents=True
)

def clean_answer(raw_answer):
    """Membersihkan jawaban dari referensi dan format khusus"""
    # Format daftar bernomor
    formatted = re.sub(r'(\d+\.)\s', r'\n\1 ', raw_answer)
    # Hapus karakter khusus
    cleaned = re.sub(r'[*_]{2}', '', formatted)
    return cleaned.strip()

def ask(query):
    try:
        # Proses query langsung tanpa validasi awal
        result = qa.invoke({"query": query})

        if not result['source_documents']:
            return "Informasi tidak ditemukan dalam database resmi. Silakan hubungi Kring Pajak 1500200"

        # Pembersihan jawaban akhir
        return clean_answer(result['result'])

    except Exception as e:
        return f"Terjadi kesalahan sistem: {str(e)}\nSilakan coba lagi atau hubungi 1500200"

# Test Query

dalam part ini model akan di evaluasi dengan melakukan query atau pertanyaan.  Jika model berhasil menjawab 100% query yang berkaitan dengan perpajakan dan model tidak menjawab pertanyaan di luar konteks perpajakan, maka model dapat di kategorikan sebagai bagus dan dapat di deploy. Jika model gagal, berarti harus kita optimasi lagi.

In [31]:
# Test Cases
questions = [
    "Saya lupa password akun DJP Online",
    "Bagaimana cara reset password?",
    "kalau saya masih pelajar apakah teteap harus bayar pajak?",
    "Cara membuat akun DJP Online baru",
    "Kalau saya kerja freelence, masuknya kategori apa?",
    "Rekomendasi restoran di Jakarta"
]

for q in questions:
    print(f"\n{'='*50}")
    print(f"Pertanyaan: {q}")
    print(f"Jawaban:\n{ask(q)}")
    print(f"{'='*50}")



Pertanyaan: Saya lupa password akun DJP Online
Jawaban:
Jika Anda lupa password akun DJP Online, Anda dapat melakukan reset password dengan langkah-langkah berikut:


1. Buka laman DJP Online.

2. Klik menu "Lupa password? Reset di sini".

3. Anda akan diminta untuk mengisi beberapa data yaitu NPWP, nomor EFIN, dan kode keamanan.

4. Setelah itu, klik "Submit".

5. Akan muncul pesan pop up "Request Succeed, link reset password telah dikirim ke email Anda". Klik "OK".

6. Cek email Anda, dan klik "Link Reset Password" yang telah dikirimkan.

7. Selanjutnya, ganti password sebelumnya dengan password baru yang aman dan mudah diingat.

Pastikan email yang Anda gunakan masih aktif dan dapat dibuka, karena link untuk reset password akan dikirimkan melalui email tersebut.

Pertanyaan: Bagaimana cara reset password?
Jawaban:
Untuk mereset password Anda, ikuti langkah-langkah berikut:


1. Buka laman DJP Online atau akun PKP Anda.

2. Klik pada menu atau link "Lupa Password?" atau "Reset Passw

In [40]:
# Test Cases
questions = [
    "kenapa pajak mahal banget",
    "kalau saya kerja di luar negri apa harus tetap bayar pajak?",
    "gua ga punya duit buat bayar pajak",
    "kalau perusahaan saya bangkrut apa harus tetap bayar pajak?",
    "Ada berapa profinsi di indonesia",
    "bagaimana cara saya lapor pajak",
    "jam operational kantor pajak jam brapa",
    "siapa nama jendral pajak sekarang"
]

for q in questions:
    print(f"\n{'='*50}")
    print(f"Pertanyaan: {q}")
    print(f"Jawaban:\n{ask(q)}")
    print(f"{'='*50}")



Pertanyaan: kenapa pajak mahal banget
Jawaban:
Pajak yang dikenakan oleh pemerintah sebenarnya tidak bisa dikategorikan sebagai mahal atau murah, karena besaran pajak ditentukan berdasarkan beberapa faktor. Beberapa faktor tersebut antara lain:


1. Tarif Pajak: Tarif pajak ditentukan oleh pemerintah dan berbeda-beda untuk setiap jenis pajak. Misalnya, untuk Pajak Penghasilan (PPh) orang pribadi, tarifnya berkisar antara 5% hingga 30% tergantung pada jumlah penghasilan.


2. Penghasilan atau Omset: Besaran pajak juga ditentukan berdasarkan penghasilan atau omset yang diterima oleh wajib pajak. Semakin besar penghasilan atau omset, maka semakin besar pula pajak yang harus dibayar.


3. Fasilitas Pajak: Pemerintah juga memberikan beberapa fasilitas pajak seperti Pajak Penghasilan Tidak Kena Pajak (PTKP) dan pengurangan pajak untuk wajib pajak tertentu.

Jadi, jika Anda merasa pajak yang Anda bayar terasa mahal, bisa jadi karena penghasilan atau omset Anda cukup besar, atau mungkin Anda 

## Model Evaluation



Dari pengujian dengan berbagai pertanyaan—baik terkait teknis akun pajak, prosedur pelaporan, kategori pajak, hingga pertanyaan non-pajak—chatbot mampu memberikan jawaban yang sangat relevan, terstruktur, dan mudah dipahami untuk semua pertanyaan yang memang tercakup dalam hasil scrapping atau konteks perpajakan Indonesia. Untuk pertanyaan seperti "Saya lupa password akun DJP Online", "Bagaimana cara reset password?", atau "Cara membuat akun DJP Online baru", sistem memberikan instruksi langkah demi langkah yang jelas, sesuai FAQ dan praktik resmi DJP. Untuk pertanyaan umum seperti "kalau saya masih pelajar apakah tetap harus bayar pajak?" atau "kalau saya kerja freelance, masuknya kategori apa?", jawaban yang diberikan juga akurat, menyesuaikan dengan ketentuan perpajakan terbaru. Sementara untuk pertanyaan di luar konteks seperti "Apa itu Mobile Legend?" atau "Rekomendasi restoran di Jakarta", chatbot secara konsisten menolak dengan sopan dan profesional, sesuai instruksi pada prompt. Secara keseluruhan, sistem mampu menjaga konsistensi, relevansi, dan profesionalisme jawaban tanpa hallucination. Overall, chat bot berjalan sesuai objective kita.

## Conclusion

Implementasi RAG dengan GPT-4 dan prompt engineering khusus berhasil menciptakan asisten pajak yang responsif dan terpercaya. Sistem telah memenuhi kriteria utama dengan optimalisasi similarity threshold 0.78 dan integrasi database MongoDB. Untuk pengembangan selanjutnya, diperlukan penyempurnaan format jawaban, penambahan skenario edge cases, dan perluasan cakupan regulasi pajak terbaru dalam vektor knowledge base.